# Lab14-2 GAN
110065508 李丞恩

In [1]:
SAMPLE_COL = 8
SAMPLE_ROW = 8
SAMPLE_NUM = SAMPLE_COL * SAMPLE_ROW

IMG_H = 64
IMG_W = 64
IMG_C = 3
IMG_SHAPE = (IMG_H, IMG_W, IMG_C)

BATCH_SIZE = 200
Z_DIM = 128
BZ = (BATCH_SIZE, Z_DIM)
BUFFER_SIZE = 20000

W_LR = 1e-4
W_EPOCH = 256

ITEMS_PER_FILE = 2000

In [2]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import os

import tensorflow as tf
import tensorflow.keras as keras
import imageio
import moviepy.editor as mpy
import IPython.display as display
import matplotlib.pyplot as plt
import pathlib
import random
import re
from pprint import pprint
from tqdm.notebook import tqdm
from tensorflow.keras import utils, datasets, layers, models
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.utils import shuffle

2021-12-22 03:43:50.866336: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [3]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # disable warnings and info

In [4]:
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)
tf.config.experimental.set_virtual_device_configuration(gpus[0], \
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit = 10000)])

2021-12-22 03:43:51.858890: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-12-22 03:43:51.890466: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-22 03:43:51.890933: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3060 computeCapability: 8.6
coreClock: 1.852GHz coreCount: 28 deviceMemorySize: 11.77GiB deviceMemoryBandwidth: 335.32GiB/s
2021-12-22 03:43:51.890951: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-12-22 03:43:51.892681: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2021-12-22 03:43:51.892715: I tensorflow/stream_executor/pl

In [5]:
if not os.path.exists("input") : os.mkdir("input")
if not os.path.exists("output") : os.mkdir("output")
if not os.path.exists("output/gif") : os.mkdir("output/gif")
if not os.path.exists("output/imgs_HW") : os.mkdir("output/imgs_HW")
if not os.path.exists("output/imgs_not_HW") : os.mkdir("output/imgs_not_HW")

## 一. 製作CelebA的tfrecord檔

### 1. Write tfrecords

In [6]:
data_root = pathlib.Path('./input/datalab-lab-14-2')
all_image_paths = list(data_root.glob('*'))
all_image_paths = [str(path) for path in all_image_paths]
all_image_paths = [path for path in all_image_paths if path[-3:] in ('png')]
image_count = len(all_image_paths)
print('\ntotal img num:', image_count)


total img num: 202599


In [7]:
DATA_PATH = './input/datalab-lab-14-2/'

def load_img(img_name):
    raw_img = tf.io.read_file(img_name) # tf.io.read_file(DATA_PATH + img_name)
    return raw_img

In [8]:
def load_img(img_name):
    raw_img = tf.io.read_file('./' + img_name)
    return raw_img

def _bytes_feature(value): # Returns a bytes_list from a string / byte.
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def to_tfrecord(img):  
    feature = {
        'img': _bytes_feature(img)
    }
    return tf.train.Example(features=tf.train.Features(feature=feature)).SerializeToString()

def tf_serialize_example(img):
    tf_string = tf.py_function(
        to_tfrecord,
        [img],
        tf.string)
    return tf.reshape(tf_string, ())

def prepare_tfrecords(all_image_paths):
    num = 0
    for i in range(0, len(all_image_paths), ITEMS_PER_FILE):
        raw_image_data = tf.data.Dataset.from_tensor_slices(all_image_paths[i:i+ITEMS_PER_FILE])
        raw_image_data = raw_image_data.map(load_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
        serialized_features_dataset = raw_image_data.map( \
            tf_serialize_example, num_parallel_calls=tf.data.experimental.AUTOTUNE)
        filename = f'./input/tfrecord/train_{num:03d}_.tfrecord'
        writer = tf.data.experimental.TFRecordWriter(filename)
        writer.write(serialized_features_dataset)
        num+=1

In [9]:
# prepare_tfrecords(all_image_paths) # 跑過一次就不要再跑了

### 2. preprocess the data

In [10]:
def processing(raw_img):
    img = tf.image.decode_png(raw_img, channels=3)
    img = tf.image.resize(img, (IMG_H, IMG_W))
    img = tf.cast(img,tf.float32)
    img = img / 255.
    return img

In [11]:
feature_description = {
    'img': tf.io.FixedLenFeature([], tf.string),
}

def _parse_function(example_proto):
    parsed = tf.io.parse_single_example(example_proto, feature_description)
    return parsed['img']

### 3. Read dataset

In [12]:
data_root = pathlib.Path('./input/tfrecord')
filename_list = list(data_root.glob('*'))
filename_list = [str(path) for path in filename_list]
filename_list = [path for path in filename_list if path[-8:] in ('tfrecord')]
tfrecord_count = len(filename_list)
print('\ntotal tfrecord num:', tfrecord_count)


total tfrecord num: 102


In [13]:
raw_dataset_train = tf.data.TFRecordDataset(filename_list)

2021-12-22 03:43:52.601028: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-22 03:43:52.601472: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-22 03:43:52.601985: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3060 computeCapability: 8.6
coreClock: 1.852GHz coreCount: 28 deviceMemorySize: 11.77GiB deviceMemoryBandwidth: 335.32GiB/s
2021-12-22 03:43:52.602059: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative va

In [14]:
parsed_dataset_train = raw_dataset_train.map(_parse_function)
dsTrain = parsed_dataset_train.map(processing)
dsTrain = dsTrain.shuffle(BUFFER_SIZE)
dsTrain = dsTrain.batch(BATCH_SIZE,drop_remainder=True)
dsTrain = dsTrain.prefetch(500)

## 二. 設計GAN的架構

In [15]:
def GAN(img_shape, z_dim):
    # x-shape
    xh, xw, xc = img_shape
    # z-shape
    zh = xh // 4
    zw = xw // 4
        
    # return Generator and Discriminator
    return keras.Sequential([ # Generator
        keras.layers.Dense(units  =  1024, input_shape = (z_dim,)),
        keras.layers.BatchNormalization(),
        keras.layers.ReLU(),
        keras.layers.Dense(units  =  zh * zw << 8), # zh * zw * 256
        keras.layers.BatchNormalization(),
        keras.layers.ReLU(),
        keras.layers.Reshape(target_shape = (zh, zw, 256)),
        keras.layers.Conv2DTranspose(
            filters = 32,
            kernel_size = 5,
            strides = 2,
            padding = "SAME"
        ),
        keras.layers.BatchNormalization(),
        keras.layers.ReLU(),
        keras.layers.Conv2DTranspose(
            filters = xc,
            kernel_size = 5,
            strides = 2,
            padding = "SAME",
            activation = keras.activations.sigmoid
        ),
    ]), keras.Sequential([ # Discriminator
        keras.layers.Conv2D(
            filters = 32,
            kernel_size = 5,
            strides = (2, 2),
            padding = "SAME",
            input_shape = img_shape,
        ),
        keras.layers.LeakyReLU(),
        keras.layers.Conv2D(
            filters = 128,
            kernel_size = 5,
            strides = (2, 2),
            padding = "SAME"
        ),
        keras.layers.BatchNormalization(),
        keras.layers.LeakyReLU(),
        keras.layers.Flatten(),
        keras.layers.Dense(units  =  1024),
        keras.layers.BatchNormalization(),
        keras.layers.LeakyReLU(),
        keras.layers.Dense(units  =  1),
    ])

s = tf.random.normal([SAMPLE_NUM, Z_DIM])

In [16]:
IWG, IWD = GAN(IMG_SHAPE, Z_DIM)
optimizer_g = keras.optimizers.Adam(W_LR, beta_1=0, beta_2=0.9)
optimizer_d = keras.optimizers.Adam(W_LR, beta_1=0, beta_2=0.9)

## 三. GAN的訓練函數定義

In [17]:
@tf.function
def IWGTrain(c1):
    z = tf.random.normal(BZ) # sample latent variable z form p(z), I don't know why but just keep it out
    with tf.GradientTape() as tpg:
        with tf.GradientTape() as tp_gradientpenalty:
            x = c1 # sample x from \mathbb{P}_r
            epsilon = np.random.uniform(0, 1) # sample \epsilon from U[0,1]
            
            x_tilde = IWG(z, training = True) # \tilde{x}<-G_\theta(z)
            x_hat = tf.math.scalar_mul(epsilon, x) + tf.math.scalar_mul((1 - epsilon), x_tilde)
            
            Dwx_tilde = IWD(x_tilde, training = True)
            Dwx = IWD(x, training = True)
            grad = IWD(x_hat, training = True)
            penalty = 10 * tf.math.square(tf.norm(tp_gradientpenalty.gradient(grad, x_hat), ord='euclidean') - 1)
            loss = Dwx_tilde - Dwx + penalty
            
            ld = tf.reduce_mean(loss)
            lg = - tf.reduce_mean(Dwx_tilde)
        
    gradient_g = tpg.gradient(lg, IWG.trainable_variables)
    optimizer_g.apply_gradients(zip(gradient_g, IWG.trainable_variables))
    return lg, ld

@tf.function
def IWDTrain(c1):
    z = tf.random.normal(BZ) # sample latent variable z form p(z), I don't know why but just keep it out
    with tf.GradientTape() as tpd:
        with tf.GradientTape() as tp_gradientpenalty:
            x = c1 # sample x from \mathbb{P}_r
            epsilon = np.random.uniform(0, 1) # sample \epsilon from U[0,1]
            
            x_tilde = IWG(z, training = True) # \tilde{x}<-G_\theta(z)
            x_hat = tf.math.scalar_mul(epsilon, x) + tf.math.scalar_mul((1 - epsilon), x_tilde)
            
            Dwx_tilde = IWD(x_tilde, training = True)
            Dwx = IWD(x, training = True)
            grad = IWD(x_hat, training = True)
            penalty = 10 * tf.math.square(tf.norm(tp_gradientpenalty.gradient(grad, x_hat), ord='euclidean') - 1)
            loss = Dwx_tilde - Dwx + penalty
            
            ld = tf.reduce_mean(loss)
            lg = - tf.reduce_mean(Dwx_tilde)
        
    gradient_d = tpd.gradient(ld, IWD.trainable_variables)
    optimizer_d.apply_gradients(zip(gradient_d, IWD.trainable_variables))
    return lg, ld

In [18]:
# ratio of training step D:G = 5:1
WTrain = (
    IWDTrain,
    IWDTrain,
    IWDTrain,
    IWDTrain,
    IWDTrain,
    IWGTrain
)

WCritic = len(WTrain)

## 四. 訓練過程視覺化

In [19]:
# Utility function
def utPuzzle(imgs, row, col, path=None):
    h, w, c = imgs[0].shape
    out = np.zeros((h * row, w * col, c), np.uint8)
    for n, img in enumerate(imgs):
        j, i = divmod(n, col)
        out[j * h : (j + 1) * h, i * w : (i + 1) * w, :] = img
    if path is not None : imageio.imwrite(path, out)
    return out
  
def utMakeGif(imgs, fname, duration):
    n = float(len(imgs)) / duration
    clip = mpy.VideoClip(lambda t : imgs[int(n * t)], duration = duration)
    clip.write_gif(fname, fps = n)

In [ ]:
wlg = [None] * W_EPOCH #record loss of g for each epoch
wld = [None] * W_EPOCH #record loss of d for each epoch
wsp = [None] * W_EPOCH #record sample images for each epoch

rsTrain = float(BATCH_SIZE) / float(image_count)
ctr = 0
for ep in tqdm(range(W_EPOCH)):
    lgt = 0.0
    ldt = 0.0
    for c1 in dsTrain:
        lg, ld = WTrain[ctr](c1)
        ctr += 1
        lgt += lg.numpy()
        ldt += ld.numpy()
        if ctr == WCritic : ctr = 0
    wlg[ep] = lgt * rsTrain
    wld[ep] = ldt * rsTrain
    
    out = IWG(s, training = False)
    img = utPuzzle(
        (out * 255.0).numpy().astype(np.uint8),
        SAMPLE_COL,
        SAMPLE_ROW,
        "./output/imgs_HW/iw_%04d.png" % ep
    )
    wsp[ep] = img
    if (ep+1) % 32 == 0:
        
        plt.imshow(img[..., 0], cmap = "gray")
        plt.axis("off")
        plt.title("Epoch %d" % ep)
        plt.show()

  0%|          | 0/256 [00:00<?, ?it/s]

2021-12-22 03:43:53.431964: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-12-22 03:43:53.456283: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2592000000 Hz
2021-12-22 03:43:57.846794: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2021-12-22 03:43:58.785493: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8100
2021-12-22 03:43:59.911972: E tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2021-12-22 03:43:59.912006: W tensorflow/stream_executor/gpu/asm_compiler.cc:56] Couldn't invoke ptxas --version
2021-12-22 03:43:59.912709: E tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2021-12-22 03:43:59.912767: W tensorflow/stream_executor/gpu/redzone_allocator.cc:31

In [ ]:
utMakeGif(np.array(wsp), "./output/gif/improved_wgan_celebA_110065508.gif", duration = 2)

In [ ]:
plt.plot(range(W_EPOCH), wld, color = "blue", label = "Discriminator Loss")
plt.plot(range(W_EPOCH), wlg, color = "red", label = "Generator Loss")
plt.legend(loc = "upper right")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Improved WGAN Training Loss")
plt.show()